In [1]:
import torch
import numpy as np
import phi.torch.flow as ptf

device = torch.device('cuda')

In [2]:
BOUNDS = ptf.Box(x=1, y=1, z=1)
RESOLUTION = 100
INFLOW = ptf.CenteredGrid(
    values=ptf.Sphere(center=ptf.tensor([0.5, 0, 0.5], ptf.channel(vector='x,y,z')), radius=0.05),
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)

In [3]:
@ptf.jit_compile
def step(v, s, dt, ir, inflow):
    s = ptf.advect.mac_cormack(s, v, dt) + ir * ptf.resample(inflow, to=s, soft=True)
    return s


delta_time = 0.2
inflow_rate = 0.2

In [4]:
index0 = 0
den0_np = np.load(f'../ret/phiflow_{index0:03d}.npz')['density']
den0_gpu = torch.tensor(den0_np, dtype=torch.float32, device=device)
DENSITY0 = ptf.CenteredGrid(
    values=ptf.wrap(den0_gpu, ptf.spatial('x,y,z')),
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
np.save(f'resim/den_{index0:03d}.npy', DENSITY0.numpy())

In [5]:
import tqdm

DENSITY = DENSITY0
for iter in tqdm.trange(119):
    vel_np = np.load(f'reconstructed/vel_{iter:03d}.npz.npy')
    vel_gpu = torch.tensor(vel_np, dtype=torch.float32, device=device)
    VELOCITY = ptf.CenteredGrid(
        values=ptf.wrap(vel_gpu, ptf.spatial('x,y,z'), ptf.channel(vector='x,y,z')),
        extrapolation=ptf.extrapolation.ZERO,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )

    ret = step(VELOCITY, DENSITY, delta_time, inflow_rate, INFLOW)
    np.save(f'resim/den_{iter + 1:03d}.npy', ret.numpy())
    DENSITY = ret

100%|██████████| 119/119 [00:05<00:00, 20.66it/s]
